# Creando Interfaces de PDI con Gradio

Este cuaderno te guiará paso a paso para crear interfaces interactivas para tus aplicaciones de Procesamiento Digital de Imágenes (PDI) utilizando la librería Gradio.

1. ¿Por qué Gradio?
Gradio es una herramienta increíblemente útil para construir rápidamente interfaces web para tus modelos o funciones de Python. En PDI, te permite:

* Visualizar tus algoritmos: Subi una imagen, aplica tu filtro y observá el resultado al instante.
* Prototipar de forma ágil: Probá diferentes ideas sin escribir código complejo de interfaz de usuario.
* Compartir tus proyectos: Genera un enlace público para que otros puedan usar tu aplicación de PDI.

2. Preparando el Entorno: Instalación de Librerías
Primero, necesitamos instalar las librerías necesarias. Ejecuta la siguiente celda para instalar Gradio, OpenCV y NumPy.

In [10]:
%%capture
!pip install gradio opencv-python numpy matplotlib
print("Librerías instaladas correctamente.")

3. Conceptos Clave de Gradio
Gradio es muy intuitivo. Los componentes principales que usaremos son:

*   `gr.Interface`: La clase principal para crear nuestra interfaz. Le decimos qué función Python ejecutar, qué tipo de entradas y salidas tiene, y le damos un título y descripción.
*   Componentes de Entrada/Salida:
    *   `gr.Image`: Para subir, mostrar y manipular imágenes. Cuando lo uses como entrada, asegúrate de poner `type="numpy"` para que tu función reciba la imagen como un array de NumPy (el formato que usa OpenCV).
    *   `gr.Slider`: Para ajustar valores numéricos (como umbrales o tamaños de kernel).
    *   `gr.Textbox`: Para entrada o salida de texto.
*   `.launch()`: El método que inicia tu aplicación web. Cuando lo ejecutes en Colab, Gradio te dará un enlace público.

4. Ejemplos Prácticos de PDI con Gradio
Vamos a ver algunos ejemplos clásicos de PDI y cómo integrarlos con Gradio. Asegúrate de ejecutar cada celda de código.

Ejemplo 4.1: Conversión a Escala de Grises
Este es nuestro "Hola Mundo" de PDI con Gradio. Sube cualquier imagen y la convertirá a escala de grises.

In [11]:
import gradio as gr
import cv2
import numpy as np

In [12]:
def convertir_a_gris(imagen_entrada):
    """
    Convierte una imagen a color (BGR) a escala de grises.
    Gradio pasa la imagen como un array de NumPy.
    """
    if imagen_entrada is None:
        return None

    # Gradio pasa la imagen como un array de NumPy (alto, ancho, canales)
    # OpenCV espera el formato (alto, ancho) para imágenes en escala de grises.
    imagen_gris = cv2.cvtColor(imagen_entrada, cv2.COLOR_BGR2GRAY)
    return imagen_gris

## Ejemplo 4.2: Aplicar un Filtro Gaussiano Ajustable

Aquí, introduciremos un `gr.Slider` para que puedas controlar el nivel de suavizado del filtro Gaussiano. El tamaño del kernel debe ser un número impar.

In [13]:
def aplicar_filtro_gaussiano(imagen_entrada, ksize_val=5):
    """
    Aplica un filtro Gaussiano a una imagen.
    ksize_val: valor del slider, lo ajustamos para que sea impar.
    """
    if imagen_entrada is None:
        return None

    # Aseguramos que el tamaño del kernel sea impar para OpenCV
    ksize = int(ksize_val)
    if ksize % 2 == 0:
        ksize += 1 # Si es par, lo convertimos a impar

    # Aplicamos el filtro Gaussiano
    imagen_suavizada = cv2.GaussianBlur(imagen_entrada, (ksize, ksize), 0)
    return imagen_suavizada

## Ejemplo 4.3: Detección de Bordes con Canny

Este ejemplo utiliza dos `gr.Slider` para controlar los umbrales del algoritmo de Canny, un detector de bordes muy popular.

In [14]:
def detectar_bordes_canny(imagen_entrada, umbral1=100, umbral2=200):
    """
    Detecta bordes en una imagen usando el algoritmo de Canny.
    Umbral1 y Umbral2: los dos umbrales para Canny.
    """
    if imagen_entrada is None:
        return None

    # Canny necesita una imagen en escala de grises.
    # Verificamos si la imagen de entrada ya es en escala de grises (2 dimensiones)
    # o si es a color (3 dimensiones, con canales).
    if len(imagen_entrada.shape) == 3:
        imagen_gris = cv2.cvtColor(imagen_entrada, cv2.COLOR_BGR2GRAY)
    else:
        imagen_gris = imagen_entrada # Ya es gris

    # Aplicamos el detector de bordes Canny
    bordes = cv2.Canny(imagen_gris, umbral1, umbral2)
    return bordes

## 5. Tu Turno: ¡Desafío de Codificación!

Ahora que viste cómo funciona, es tu turno de crear tu propia interfaz.

**Desafío:**

Implementa una función de PDI que invierta los colores de una imagen (negativo). Luego, crea una interfaz de Gradio para esta función.

**Pistas:**

*   Para invertir los colores de una imagen en formato NumPy (que Gradio te da), simplemente resta el valor máximo de píxel (255 para imágenes de 8 bits) al valor actual de cada píxel. Ejemplo: `imagen_invertida = 255 - imagen_entrada`.
*   Asegúrate de que tu función maneje correctamente los casos de imágenes en escala de grises y a color, o que se enfoque solo en uno y lo documente.

In [16]:
def invertir_colores(imagen_entrada):

    if imagen_entrada is None:
        return None
    imagen_invertida = 255 - imagen_entrada
    return imagen_invertida


In [22]:
# invertir_colores
# Detector de Bordes Canny Interactivo 🔍

def reset_image(original: np.ndarray) -> np.ndarray:
    return original

with gr.Blocks() as demo:
    original_image = gr.State(value=None)  # Guarda la imagen original

    input_img = gr.Image(type="numpy", label="Imagen original")
    output_img = gr.Image(type="numpy", label="Imagen transformada", elem_classes="fixed-img")

    btn1 = gr.Button("Escala de grises")
    btn2 = gr.Button("Filtro Gaussiano")
    btn3 = gr.Button("Detectar bordes")
    btn4 = gr.Button("Invertir colores")
    btn_reset = gr.Button("Reset")

    # Cuando se carga una nueva imagen, la guardamos y la mostramos
    def store(img):
        return img, img

    input_img.change(store, inputs=[input_img], outputs=[output_img, original_image])

    btn1.click(convertir_a_gris, inputs=[output_img], outputs=[output_img])
    btn2.click(aplicar_filtro_gaussiano, inputs=[output_img], outputs=[output_img])
    btn3.click(detectar_bordes_canny, inputs=[output_img], outputs=[output_img])
    btn4.click(invertir_colores, inputs=[output_img], outputs=[output_img])
    #invertir_colores=[original_image], outputs=[output_img])

    # Al cargar una imagen, se ejecuta store():
    btn_reset.click(reset_image, inputs=[original_image], outputs=[output_img])


demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2bb2c50bdb680ae9c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
# Crea tu interfaz de Gradio aquí
# Creamos la interfaz de Gradio
# fn: la función de PDI que se ejecutará
# inputs: el tipo de componente de entrada (una imagen)
# outputs: el tipo de componente de salida (una imagen)
interfaz_gris = gr.Interface(
    fn=invertir_colores,
    inputs=gr.Image(type="numpy", label="Subí una Imagen a Color"),
    outputs=gr.Image(type="numpy", label="Imagen en Escala de Grises"),
    title="⭐ Invertir colores ⭐",
    description="Subí una imagen a color para verla convertida a escala de grises al instante."
)

# Lanzamos la interfaz. Haz clic en el enlace público (Running on public URL)
# para que puedas compartirlo con quien quieras.
#interfaz_gris.launch()

## 6. Más Allá: ¿Qué sigue?

Gradio ofrece muchas más opciones y componentes. Una vez que te sientas cómodo con lo básico, puedes explorar:

*   `gr.Blocks`: Para diseños de interfaz más complejos y personalizados.
*   `gr.Tabs`: Para organizar múltiples funcionalidades en pestañas.
*   `gr.State`: Para mantener el estado entre interacciones del usuario.
*   Compartir en Hugging Face Spaces: Despliega tus aplicaciones de Gradio de forma gratuita y permanente.

¡Espero que este cuaderno te haya dado una base sólida para empezar a crear interfaces increíbles para tus proyectos de PDI! Si tienes preguntas, no dudes en consultar la [documentación oficial de Gradio](https://www.gradio.app/docs/gradio/interface).

# Apropiación del cuaderno de Gradio

Este cuaderno fue intervenido para hacer más dinamica la transformación de la imagen original, a la cual se le puede invertir los colores, y despues ver la imagen en escala de grises, aplicar un filtro gaussiano, y despues volver a invertir los colores.

El uso de gradio permite poner a prueba las diferentes transformaciones que vimos en clase  y poder compartirlo.